# Benchmarking methods

This document provides a comprehensive guide to benchmarking different imputation methods using MicroImpute. The examples below illustrate the workflow for comparing various imputation approaches and evaluating their performance.

In [11]:
# On the Diabetes Dataset

from typing import List, Type

import pandas as pd

from microimpute.comparisons import *
from microimpute.config import RANDOM_STATE
from microimpute.models import *
from microimpute.visualizations.plotting import method_comparison_results
from microimpute.utils.data import preprocess_data
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore")

# 1. Prepare data
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=RANDOM_STATE
)

predictors = ["age", "sex", "bmi", "bp"]
imputed_variables = ["s1", "s4"]

Y_test: pd.DataFrame = X_test[imputed_variables]

# 2. Run imputation methods
model_classes: List[Type[Imputer]] = [QRF, OLS, QuantReg, Matching]
method_imputations = get_imputations(
    model_classes, X_train, X_test, predictors, imputed_variables
)

# 3. Compare imputation methods
loss_comparison_df = compare_quantile_loss(
    Y_test, method_imputations, imputed_variables
)

# 4. Plot results
comparison_viz = method_comparison_results(
    data=loss_comparison_df,
    metric_name="Test Quantile Loss",
    data_format="long",
)
fig = comparison_viz.plot(
    title="Method Comparison on Diabetes Dataset",
    show_mean=True,
)
fig.show()

Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.
Found 1 numeric columns with unique values < 10, treating as categorical: ['sex']. Converting to dummy variables.


In [12]:
# On the SCF Dataset

from typing import List, Type, Optional, Union

import io
import logging
import pandas as pd
from pydantic import validate_call
import requests
import zipfile

from microimpute.comparisons import *
from microimpute.config import RANDOM_STATE, VALIDATE_CONFIG, VALID_YEARS
from microimpute.models import *
from microimpute.utils.data import preprocess_data

import warnings

logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")

# 1. Prepare data
@validate_call(config=VALIDATE_CONFIG)
def load_scf(
    years: Optional[Union[int, List[int]]] = None,
    columns: Optional[List[str]] = None,
) -> pd.DataFrame:
    """Load Survey of Consumer Finances data for specified years and columns.

    Args:
        years: Year or list of years to load data for.
        columns: List of column names to load.

    Returns:
        DataFrame containing the requested data.

    Raises:
        ValueError: If no Stata files are found in the downloaded zip
            or invalid parameters
        RuntimeError: If there's a network error or a problem processing
            the downloaded data
    """
    def scf_url(year: int) -> str:
        """Return the URL of the SCF summary microdata zip file for a year."""

        if year not in VALID_YEARS:
            logger.error(
                f"Invalid SCF year: {year}. Valid years are {VALID_YEARS}"
            )
            raise

        url = f"https://www.federalreserve.gov/econres/files/scfp{year}s.zip"
        return url

    logger.info(f"Loading SCF data with years={years}")

    try:
        # Identify years for download
        if years is None:
            years = VALID_YEARS
            logger.warning(f"Using default years: {years}")

        if isinstance(years, int):
            years = [years]

        all_data: List[pd.DataFrame] = []

        for year in years:
            logger.info(f"Processing data for year {year}")
            try:
                # Download zip file
                logger.debug(f"Downloading SCF data for year {year}")
                url = scf_url(year)
                try:
                    response = requests.get(url, timeout=60)
                    response.raise_for_status()  # Raise an error for bad responses
                except requests.exceptions.RequestException as e:
                    logger.error(
                        f"Network error downloading SCF data for year {year}: {str(e)}"
                    )
                    raise

                # Process zip file
                z = zipfile.ZipFile(io.BytesIO(response.content))
                # Find the .dta file in the zip
                dta_files: List[str] = [
                    f for f in z.namelist() if f.endswith(".dta")
                ]
                if not dta_files:
                    logger.error(
                        f"No Stata files found in zip for year {year}"
                    )
                    raise

                # Read the Stata file
                try:
                    logger.debug(f"Reading Stata file: {dta_files[0]}")
                    with z.open(dta_files[0]) as f:
                        df = pd.read_stata(
                            io.BytesIO(f.read()), columns=columns
                        )
                        logger.debug(
                            f"Read DataFrame with shape {df.shape}"
                        )
                except Exception as e:
                    logger.error(
                        f"Error reading Stata file for year {year}: {str(e)}"
                    )
                    raise

                # Add year column
                df["year"] = year
                logger.info(
                    f"Successfully processed data for year {year}, shape: {df.shape}"
                )
                all_data.append(df)

            except Exception as e:
                logger.error(f"Error processing year {year}: {str(e)}")
                raise

        # Combine all years
        logger.debug(f"Combining data from {len(all_data)} years")
        if len(all_data) > 1:
            result = pd.concat(all_data)
            logger.info(
                f"Combined data from {len(years)} years, final shape: {result.shape}"
            )
            return result
        else:
            logger.info(
                f"Returning data for single year, shape: {all_data[0].shape}"
            )
            return all_data[0]

    except Exception as e:
        logger.error(f"Error in _load: {str(e)}")
        raise

scf_data = load_scf(2022)
PREDICTORS: List[str] = [
        "hhsex",  # sex of head of household
        "age",  # age of respondent
        "married",  # marital status of respondent
        # "kids",  # number of children in household
        "race",  # race of respondent
        "income",  # total annual income of household
        "wageinc",  # income from wages and salaries
        "bussefarminc",  # income from business, self-employment or farm
        "intdivinc",  # income from interest and dividends
        "ssretinc",  # income from social security and retirement accounts
        "lf",  # labor force status
    ]
IMPUTED_VARIABLES: List[str] = ["networth"]

X_train, X_test = preprocess_data(
    data=scf_data, full_data=False, normalize=False,
)

# Shrink down the data by sampling
X_train = X_train.sample(frac=0.01, random_state=RANDOM_STATE)
X_test = X_test.sample(frac=0.01, random_state=RANDOM_STATE)

Y_test: pd.DataFrame = X_test[IMPUTED_VARIABLES]

# 2. Run imputation methods
model_classes: List[Type[Imputer]] = [QRF, OLS, QuantReg, Matching]
method_imputations = get_imputations(
    model_classes, X_train, X_test, PREDICTORS, IMPUTED_VARIABLES
)

# 3. Compare imputation methods
loss_comparison_df = compare_quantile_loss(
    Y_test, method_imputations, IMPUTED_VARIABLES
)

# 4. Plot results
comparison_viz = method_comparison_results(
    data=loss_comparison_df,
    metric_name="Test Quantile Loss",
    data_format="long",
)
fig = comparison_viz.plot(
    title="Method Comparison on SCF Dataset",
    show_mean=True,
)
fig.show()

Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treating as categorical: ['hhsex', 'married', 'race']. Converting to dummy variables.
Found 3 numeric columns with unique values < 10, treati

## Data preparation

The data preparation phase establishes the foundation for meaningful benchmarking comparisons. The `load_scf()` function downloads data from user-specified survey years, carefully selecting relevant predictor and target variables that capture the essential relationships for imputation. 

The `preprocess_data()` applies normalization techniques to the features when `normalize=True`, ensuring that variables with different scales don't unduly influence the imputation models. This preprocessing step is crucial, particularly if introducing additional for methods like nearest neighbor matching that rely on distance calculations. Finally, the function splits the data into training and testing sets, maintaining the statistical properties of both sets while creating an appropriate evaluation framework. If you would like to normalizing the data set without splitting it (for example in the event of performing cross-validation), set the `full_data` parameter to `False`. 

```python
# Normalizing
processed_data = preprocess_data(dataset, full_data=True)

# Normalizing and splitting
X_train, X_test = preprocess_data(dataset, normalize=True, full_data=False)
```

## Imputation generation

The imputation generation process serves as the core operational phase of the benchmarking framework. The `get_imputations()` function orchestrates this process with remarkable efficiency, handling all aspects of model training and prediction generation. It systematically trains each specified model on identical training data, ensuring a fair comparison across different imputation approaches.

After training, the function generates predictions at user-specified quantiles, allowing for evaluation across different parts of the conditional distribution. The quantile-based approach provides insights not just into central tendency (as with mean-based methods) but into the entire shape of the imputed distributions. This comprehensive prediction generation creates a rich dataset for subsequent evaluation.

The function organizes all results into a consistent, structured format designed for straightforward comparison. The returned nested dictionary architecture provides intuitive access to predictions from different models at different quantiles:

```
{
    "ModelName1": {
        0.1: DataFrame of predictions at 10th percentile,
        0.5: DataFrame of predictions at 50th percentile,
        0.9: DataFrame of predictions at 90th percentile.
    },
    "ModelName2": {
        0.1: DataFrame of predictions at 10th percentile,
        ...
    },
    ...
}
```

This well-designed data structure simplifies downstream analysis and visualization, allowing researchers to focus on interpreting results rather than managing data formats.

At this stage, a model object can only handle the imputation of on variable at a time, meaning that to impute multiple variables from a data set, a new model object must be created for each of them. 

## Quantile loss calculation

The evaluation phase employs sophisticated quantile loss metrics to assess imputation quality. This approach provides a more nuanced evaluation than traditional metrics like mean squared error, particularly for capturing performance across different parts of the distribution.

At the foundation of this evaluation lies the `quantile_loss()` function, which implements the standard quantile loss formulation: 

$$L(y, f, q) = max(q \cdot (y - f), (q - 1)(y - f)),$$ 

where $q$ is the quantile to be evaluated, $y$ representes the true value and $f$ is the imputed value. 

This mathematical formulation creates an asymmetric loss function that penalizes under-prediction more heavily for higher quantiles and over-prediction more heavily for lower quantiles. This asymmetry aligns perfectly with the interpretation of quantiles—a 90th percentile prediction should rarely be below the true value, while a 10th percentile prediction should rarely exceed it.

Building on this foundation, the `compute_quantile_loss()` function calculates losses between true and imputed values, providing granular insight into model performance at the individual prediction level. This detailed evaluation helps identify specific patterns or regions where certain models might excel or struggle.

The integration of these components culminates in the `compare_quantile_loss()` function, which systematically evaluates multiple methods across different quantiles. The function produces a structured DataFrame with columns that describe the method being evaluated, the specific percentile being assessed, and the corresponding average quantile loss value.

## Visualization

The `method_comparison_results.plot()` function generates bar charts that present benchmarking results grouping results by both model and quantile, allowing quickly identifying patterns and trends in performance across different methods and different parts of the distribution.

The function employs color coding to visually distinguish between different imputation models, making it easy to track the performance of a single method. Along the horizontal axis, the chart displays different quantiles (such as the 10th, 25th, 50th percentiles), allowing assessment across the entire distribution of interest. The vertical axis represents the average quantile loss, with lower values indicating better performance—this clear metric gives an immediate visual indication of which models are performing well. The dashed lines represent average loss across quantiles.

## Extending the benchmarking framework

The MicroImpute benchmarking framework was designed with extensibility as a core principle, allowing researchers to easily integrate and evaluate new imputation approaches. To incorporate your own custom imputation model into this evaluation framework, you can follow a straightforward process.

First, implement your custom model by extending the `Imputer` abstract base class, following the design patterns and interface requirements documented in the [implement-new-model.md](../models/imputer/implement-new-model.md) file. This structured approach ensures your model will interact correctly with the rest of the benchmarking system. Once your model implementation is complete, simply include your model class in the `model_classes` list alongside the built-in models you wish to compare against. Finally, execute the benchmarking process as described previously, and your custom model will be evaluated using the same rigorous methodology applied to the built-in models.

This seamless integration is possible because all models that implement the `Imputer` interface share a common API, allowing the benchmarking framework to interact with them in a consistent manner regardless of their internal implementation details. This architectural decision makes the framework inherently extensible while maintaining a clean separation between the benchmarking logic and the specific imputation methods being evaluated.

## Best practices

Effective benchmarking requires careful attention to methodology and interpretation. To maximize the value of your imputation benchmarking efforts, consider following these research-based best practices that ensure comprehensive and reliable evaluation.

Robust evaluation requires testing models across multiple diverse datasets rather than relying on a single test case. This approach helps identify which models perform consistently well across different data scenarios and which may be sensitive to particular data characteristics. By examining performance across varied contexts, you can make more confident generalizations about a method's effectiveness.

A comprehensive evaluation should assess performance across different quantiles rather than focusing solely on central measures like the median. Many applications care about the tails of distributions, and models that perform well at the median might struggle with extreme quantiles. Evaluating across the full spectrum of quantiles provides a more complete picture of each method's strengths and limitations.

While statistical performance is critical, practical considerations should not be overlooked. Different imputation methods can vary dramatically in their computational requirements, including training time, memory usage, and prediction speed. In many applications, a slightly less accurate method that runs orders of magnitude faster may be preferable. Consider these trade-offs explicitly in your evaluation framework.

For particularly important decisions, enhance the reliability of your performance estimates through cross-validation techniques. Cross-validation provides a more stable estimate of model performance by averaging results across multiple train-test splits, reducing the impact of any particular data division. This approach is especially valuable when working with smaller datasets where a single train-test split might not be representative.

The package also supports detailed assessment of model behavior through train-test performance comparisons via the `model_performance_results()` function. This visualization tool helps identify potential overfitting or underfitting issues by contrasting a model's performance on training data with its performance on held-out test data. Significant disparities between training and testing performance can reveal important limitations in a model's generalization capabilities.

For specialized applications with particular interest in certain parts of the distribution, the framework accommodates custom quantile sets for targeted evaluation. Rather than using the default (random) quantiles, researchers can specify exactly which quantiles to evaluate, allowing focused assessment of performance in regions of particular interest. This flexibility enables tailored evaluations that align precisely with application-specific requirements and priorities.
